In [206]:
# import packages
import os
import pandas as pd
import random
import numpy as np
import scipy
# import pycaret
import datetime as dt
import json
from sodapy import Socrata

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler

NYC Data from City of New York
https://data.cityofnewyork.us/Housing-Development/Electric-Consumption-And-Cost-2010-Feb-2022-/jr24-e7cr

In [169]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("jr24-e7cr", limit=407031)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [7]:
# %store results_df

UsageError: Unknown variable 'results_df'


In [8]:
%store -r results_df

In [175]:
df = results_df.copy()

# change data types to match
df['consumption_kwh'] = df['consumption_kwh'].astype(float)
df['kwh_charges'] = df['kwh_charges'].astype(float)
df['days'] = df['days'].astype(float)

In [176]:
# select only last 5 complete years of data
df['revenue_month'] = pd.to_datetime(df['revenue_month'], format = '%Y-%m')
df = df.loc[(df['revenue_month']>='2017-01-01' )& (df['revenue_month']<'2022-01-01')]

# take only month integer
df['revenue_month'] = df['revenue_month'].dt.month

In [52]:
# count of missing values for every column
df.isna().sum()

development_name           0
borough                    0
account_name               0
location                4172
meter_amr                154
tds                     1055
edp                        0
rc_code                    0
funding_source             0
amp                      677
vendor_name                0
umis_bill_id               0
revenue_month              0
service_start_date         8
service_end_date           8
days                       8
meter_number               0
estimated                  0
current_charges            0
rate_class                 0
bill_analyzed              0
consumption_kwh            0
kwh_charges                0
consumption_kw             0
kw_charges                 0
other_charges              0
meter_scope           169785
anom_cost                  0
anom_qty                   0
has_cost_anom              0
has_qty_anom               0
dtype: int64

In [49]:
# count of all unique values for every column
df.nunique()

development_name         336
borough                    7
account_name             354
location                 488
meter_amr                  6
tds                      336
edp                      331
rc_code                  385
funding_source             7
amp                      158
vendor_name                3
umis_bill_id           49414
revenue_month             12
service_start_date       299
service_end_date         327
days                      47
meter_number            6131
estimated                  3
current_charges       136297
rate_class                14
bill_analyzed              2
consumption_kwh         5098
kwh_charges            56607
consumption_kw          9929
kw_charges             50707
other_charges         110171
meter_scope              100
anom_cost              60157
anom_qty                7428
has_cost_anom              2
has_qty_anom               2
dtype: int64

In [177]:
df['meter_amr'] = df['meter_amr'].str.replace('ENT|B-12|398|NONE', 'OTHER/NONE')
df['meter_amr'] = np.where(df['meter_amr'].isna(), 'OTHER/NONE', df['meter_amr'] )
df['funding_source'] =  np.where(df['funding_source']!='FEDERAL', 'OTHER', df['funding_source'])
df['rate_class'] = np.where(df['rate_class'].str.contains('GOV/NYC'), df['rate_class'], 'OTHER')

/var/folders/hp/1lw7zlt10ql7hhc7hv8cm8lr0000gn/T/ipykernel_34901/1795570298.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['meter_amr'] = df['meter_amr'].str.replace('ENT|B-12|398|NONE', 'OTHER/NONE')


## Add anomalies to dataset

https://towardsdatascience.com/create-synthetic-time-series-with-anomaly-signatures-in-python-c0b80a6c093c

As a synthetic data generation method, you want to control the following characteristics of the anomalies:
Fraction of the data that need to be anomalous
The scale of the anomaly (how far they lie from the normal)
One-sided or two-sided (higher or lower than the normal data in magnitude)


In [178]:
# select random rows to be anomalized - generate list of all indices and pick randomly from list
totalRows = len(df)
inds = range(totalRows)
perc = .05 # 5% of data chosen to be anomalized for both consumption and cost columns
n = totalRows * perc


# cost indices
anomalized_inds_cost = random.sample(inds, int(n))

# common typos to add to data - these also take into account adding/removing 0s by accident

cat1_cost = anomalized_inds_cost[0::4] # rows with x 10 (one decimal to the right)
cat2_cost = anomalized_inds_cost[1::4] # rows with x 100 (2 decimals to the right)
cat3_cost = anomalized_inds_cost[2::4] # rows with x .1 (one decimal point to the left)
cat4_cost = anomalized_inds_cost[3::4] # rows with x .01 (one decimal point to the left) 

df['anom_cost'] = df['kwh_charges']
df['anom_cost'].iloc[cat1_cost] = df['anom_cost'].iloc[cat1_cost] * 10
df['anom_cost'].iloc[cat2_cost] = df['anom_cost'].iloc[cat2_cost] * 100
df['anom_cost'].iloc[cat3_cost] = df['anom_cost'].iloc[cat3_cost] * .1
df['anom_cost'].iloc[cat4_cost] = df['anom_cost'].iloc[cat4_cost] * .01

# qty indices
anomalized_inds_qty = random.sample(inds, int(n))

cat1_qty = anomalized_inds_qty[0::4] # rows with x 10 (one decimal to the right)
cat2_qty = anomalized_inds_qty[1::4] # rows with x 100 (2 decimals to the right)
cat3_qty = anomalized_inds_qty[2::4] # rows with x .1 (one decimal point to the left)
cat4_qty = anomalized_inds_qty[3::4] # rows with x .01 (one decimal point to the left) 

df['anom_qty'] = df['consumption_kwh']
df['anom_qty'].iloc[cat1_qty] = df['anom_qty'].iloc[cat1_qty] * 10
df['anom_qty'].iloc[cat2_qty] = df['anom_qty'].iloc[cat2_qty] * 100
df['anom_qty'].iloc[cat3_qty] = df['anom_qty'].iloc[cat3_qty] * .1
df['anom_qty'].iloc[cat4_qty] = df['anom_qty'].iloc[cat4_qty] * .01

df['has_cost_anom'] = ''
df['has_cost_anom'] = np.where((df['anom_cost']!=df['kwh_charges']),'ANOMALY','NO ANOMALY')


df['has_qty_anom'] = ''
df['has_qty_anom'] = np.where((df['anom_qty']!=df['consumption_kwh']),'ANOMALY','NO ANOMALY')

print('Total Anomalies:',len(df.loc[(df['has_qty_anom']=='ANOMALY')|(df['has_cost_anom']=='ANOMALY')]))

del cat1_cost
del cat2_cost
del cat3_cost
del cat4_cost

del cat1_qty
del cat2_qty
del cat3_qty
del cat4_qty


Total Anomalies: 10905


/var/folders/hp/1lw7zlt10ql7hhc7hv8cm8lr0000gn/T/ipykernel_34901/3722355204.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['anom_cost'].iloc[cat1_cost] = df['anom_cost'].iloc[cat1_cost] * 10
/var/folders/hp/1lw7zlt10ql7hhc7hv8cm8lr0000gn/T/ipykernel_34901/3722355204.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['anom_cost'].iloc[cat2_cost] = df['anom_cost'].iloc[cat2_cost] * 100
/var/folders/hp/1lw7zlt10ql7hhc7hv8cm8lr0000gn/T/ipykernel_34901/3722355204.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

In [207]:
X = df[['borough', 'funding_source', 'revenue_month', 'anom_cost', 'anom_qty','meter_amr', 'rate_class']]
X = pd.get_dummies(X, columns=['funding_source', 'borough', 'meter_amr','rate_class'], prefix= ['funding', 'borough', 'meter_amr', 'rate'])


scale= StandardScaler()
 
# standardization of dependent variables
X = scale.fit_transform(X) 

In [209]:
lof  = LocalOutlierFactor(contamination=.05)
iso = IsolationForest(contamination=.05, random_state = 0)

import time
start = time.time()
df['anomaly_outcome_lof'] = lof.fit_predict(X)
df['lof_anomaly_score'] = lof.negative_outlier_factor_
# df['lof_anomaly_score_radius'] = (df['lof_anomaly_score'].max() - df['lof_anomaly_score']) / (df['lof_anomaly_score'].max() - df['lof_anomaly_score'].min())
print('LOF predictions complete')

start = time.time()
df['anomaly_outcome_iso'] = iso.fit_predict(X)
df['iso_anomaly_score'] = abs(iso.score_samples(X))
print('Iso Forest predictions complete')


df['correct_prediction_lof'] = np.where(((df['anomaly_outcome_lof'] == -1) & ((df['has_qty_anom']=='ANOMALY')|(df['has_cost_anom']=='ANOMALY')))|(df['anomaly_outcome_lof'] == 1) & ((df['has_qty_anom']=='NO ANOMALY')& (df['has_cost_anom']=='NO ANOMALY')), 'YES', 'NO')
pred_results_lof = df['correct_prediction_lof'].value_counts()

df['correct_prediction_iso'] = np.where(((df['anomaly_outcome_iso'] == -1) & ((df['has_qty_anom']=='ANOMALY')|(df['has_cost_anom']=='ANOMALY')))|(df['anomaly_outcome_iso'] == 1) & ((df['has_qty_anom']=='NO ANOMALY')& (df['has_cost_anom']=='NO ANOMALY')), 'YES', 'NO')
pred_results_iso = df['correct_prediction_iso'].value_counts()

print('LOF Model Accuracy:',round(pred_results_lof[0]/(pred_results_lof[1]+pred_results_lof[0])*100,1),'%',' in ', round(time.time() - start,1), 's', '\n', 'Iso Forest Model Accuracy:',round(pred_results_iso[0]/(pred_results_iso[1]+pred_results_iso[0])*100,1),'%', ' in ', round(time.time() - start,1), 's')

LOF predictions complete
Iso Forest predictions complete
LOF Model Accuracy: 96.0 %  in  9.6 s 
 Iso Forest Model Accuracy: 90.2 %  in  9.6 s


In [212]:
path = '/Users/viviantran/projects/jpl_interview/data/processed'
os.chdir(path)

df.to_csv('modelResults.csv', index = False)

# Misc Notes

## Anomaly Detection Algorithms to Choose From

https://towardsdatascience.com/5-anomaly-detection-algorithms-every-data-scientist-should-know-b36c3605ea16
Depends on what kind of anomalies exist in our data:
- Outliers: Short/small anomalous patterns that appear in a non-systematic way in data collection.
- Change in Events: Systematic or sudden change from the previous normal behavior.
- Drifts: Slow, undirectional, long-term change in the data.



Point anomalies – if a data point is too far from the rest, it falls into the category of point anomalies. The above example of bank transaction illustrates point anomalies.
Contextual anomalies – If the event is anomalous for specific circumstances (context), then we have contextual anomalies. As data becomes more and more complex, it is vital to use anomaly detection methods for the context. This anomaly type is common in time-series data. Example – spending $10 on ice-cream every day during the hot months is normal, but is odd for the rest months.
Collective anomalies. The collective anomaly denotes a collection of anomalous with respect to the whole dataset, but not individual objects. Example: breaking rhythm in ECG (Electrocardiogram).



We are not using simple statistical methods like comparing mean and median because our data is not univariate. Typos/errors can occur in the cost or consumption columns, so we need to choose a method that can take this into account. In addition, natural dips and peaks that are typical for that account could be marked as outliers if we were to do simple comparisons. A machine learning approach will take into account the historical context. 

Simple statistical techniques such as mean, median, quantiles can be used to detect univariate anomalies feature values in the dataset. Various data visualization and exploratory data analysis techniques can be also be used to detect anomalies.

This project originated from my work with naval utility data, which contained electricity, water, natural gas, and sewer along with over 4000 different accounts and line item descriptions to take into account. Since this data is sensitive, I've recreated the assignment using NYC utility data, which only contains electricity and about 300 accounts. The NYC data is also cleaned an doesn't contain any errors, so I artificially introduced some errors into the data (approx 5% of the data will contain errors). The original dataset where I was working with all naval utilities worldwide had much more errors because we dealt with a lot of manual data entry. We also had bill corrections which this data does not contain. This data comes directly from the electricity meter. 



Checklist:
1. Isolation Forest
   1. Chose isolation forest because it is easy to interpret for our stakeholders
2. Local Outlier Factor
3. Robust Covariance
4. One-Class SVM
5. One-Class SVM (SGD)

https://www.intellspot.com/anomaly-detection-algorithms/

More Anomaly Info:
https://serokell.io/blog/anomaly-detection-in-machine-learning
- We are dealing with contextual outliers

Requirements:
- Unsupervised: don't actually know what is is/isnt an error
- Flexible/robust enough to account for data changes
- Interpretable

Looking at only variance and standard deviations assumes that all groups will behave the same and doesn't take into account context of the account
Rule based analysis where we are looking at vaue thresholds assume we know what an acceptable range is. What happens when we have new data? How we take that into account? Do we have to create rules for every single group? Unwieldy for large datasets and not sustainable in the long-run

The k-NN algorithm works very well for dynamic environments where frequent updates are needed. In addition, density-based distance measures are good solutions for identifying unusual conditions and gradual trends. This makes k-NN useful for outlier detection and defining suspicious events.
k-NN also is very good techniques for creating models that involve non-standard data types like text.

k-NN is one of the proven anomaly detection algorithms that increase the fraud detection rate. It is also one of the most known text mining algorithms out there.


The LOF is a key anomaly detection algorithm based on a concept of a local density. It uses the distance between the k nearest neighbors to estimate the density.

LOF compares the local density of an item to the local densities of its neighbors. Thus one can determine areas of similar density and items that have a significantly lower density than their neighbors. These are the outliers.


K-means clustering
- Only works with numeric data - we have some categorical cols

Time series
- Not using this because we are taking into account more than just the numeric values. We want to feed in the vendor, location, rate type because anomalies could be specific to groups within these variables




https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf - iso tree paper

Anomalies are more susceptible to isolation and hence have short path lengths (used to calculate the anomaly score)

calculated iso tree anomaly scores:
- Very close to 1 = definitely anomalies
- much smaller than 0.5, then definitely normal
- all instances ~0.5, then sample doesn't have distinct anomaly


LOF
raw scores: larger scores = more likely to be outlier
LOF measures how isolated a point is from its surrounding neighbors 


isotree output easier to interpret 




clf.negative_outlier_factor_



https://medium.com/@arpitbhayani/isolation-forest-algorithm-for-anomaly-detection-f88af2d5518d